In [5]:
# Importing Dataset
import pandas as pd
dataset=pd.read_csv('Churn_Modelling.csv')

# Spliting input and output attributes
array=dataset.values
x=array[:,3:-1]
y=array[:,-1]

# Encode catergorical values
from sklearn.preprocessing import LabelEncoder
labelencoder_1=LabelEncoder()
x[:,1]=labelencoder_1.fit_transform(x[:,1])
labelencoder_2=LabelEncoder()
x[:,2]=labelencoder_2.fit_transform(x[:,2])

# Encoding numerical values as dummy variables
from sklearn.preprocessing import OneHotEncoder
onehotencoder=OneHotEncoder(categorical_features=[1])
x=onehotencoder.fit_transform(x).toarray()

# Handling Dummy variable trap
x=x[:,1:]

# Normalization Scale
from sklearn.preprocessing import Normalizer
scaled_value=Normalizer()
x=scaled_value.fit_transform(x)

# Split the dataset into train and test set
train_size=0.80
test_size=0.20
seed=5
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=train_size,test_size=test_size,random_state=seed)

C:\Users\arunkumar\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\arunkumar\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
# Evalute The ANN Algrithm using KFold Technique to minimize variance
# Below library we are using to make connect betwenn keras framework to scikit_learn library
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
nodes=len(x_train[0])
halfnode=int(nodes+1/2)
def build_ann():
    ann=Sequential()
    ann.add(Dense(output_dim=halfnode,input_dim=nodes,activation='relu',init='uniform'))
    ann.add(Dense(output_dim=halfnode,init='uniform',activation='relu'))
    ann.add(Dense(output_dim=halfnode,init='uniform',activation='relu'))
    ann.add(Dense(output_dim=1,activation='sigmoid',init='uniform'))
    ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return ann
ann=KerasClassifier(build_fn=build_ann,batch_size=10,nb_epoch=100)
predictions=cross_val_score(estimator=ann,X=x_train,y=y_train,cv=10,n_jobs=-1)
mean_Accuracy=predictions.mean()
variance_mean_accuracy=predictions.std()
print(predictions)
print(mean_Accuracy)
print(variance_mean_accuracy)

[0.8075     0.7925     0.8175     0.76375    0.79875    0.80624999
 0.78125    0.775      0.81125    0.80625   ]
0.7959999971464276
0.016600074941073958


In [7]:
# Improve Ann Algorithm from over and underfitting to trainset using dropout class


# dropout class if used to disable certain nodes while training the model it range from 0 to 1 means 0.1=10% to 1=100%
# we need to use in between 0.1 to 0.4 value if we use above 0.5 it will be under fit.

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import cross_val_score
nodes=len(x_train[0])
halfnode=int(nodes+1/2)
def build_ann():
    ann=Sequential()
    ann.add(Dense(output_dim=halfnode,input_dim=nodes,init='uniform',activation='relu'))
    # It will drop 10% nodes in a layer to prevent overfitting
    ann.add(Dropout(p=0.1))
    ann.add(Dense(output_dim=halfnode,init='uniform',activation='relu'))
    ann.add(Dropout(p=0.1))
    ann.add(Dense(output_dim=halfnode,init='uniform',activation='relu'))
    ann.add(Dropout(p=0.1))
    ann.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return ann
ann=KerasClassifier(build_fn=build_ann,batch_size=10,nb_epoch=100)
predictions=cross_val_score(estimator=ann,X=x_train,y=y_train,cv=10,n_jobs=-1)
mean_predictions=predictions.mean()
mean_variance_predicitons=predictions.std()
print(predictions)
print(mean_predictions)
print(mean_variance_predicitons)

[0.8075     0.7925     0.8175     0.76375    0.79875    0.80624999
 0.78125    0.775      0.81125    0.80625   ]
0.7959999971464276
0.016600074941073958


In [ ]:
# Tunning ANN Algorithm to check hyperparameter to find best combination
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
nodes=len(x_train[0])
half=int(nodes+1/2)
def build_ann(optimizer):
    ann=Sequential()
    ann.add(Dense(output_dim=half,input_dim=nodes,init='uniform',activation='relu'))
    ann.add(Dropout(p=0.2))
    ann.add(Dense(output_dim=half,init='uniform',activation='relu'))
    ann.add(Dropout(p=0.2))
    ann.add(Dense(output_dim=half,init='uniform',activation='relu'))
    ann.add(Dropout(p=0.2))
    ann.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    ann.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return ann
ann=KerasClassifier(build_fn=build_ann)
params={'optimizer':['adam','rmsprop'],
        'batch_size':[8,16,32],
        'nb_epoch':[50,100,150,200]}
results=GridSearchCV(estimator=ann,
                    param_grid=params,
                    cv=10)
grid_search=results.fit(x_train,y_train)
best_parameter=grid_search.best_params_
best_accuracy=grid_search.best_score_
print(best_parameter)
print(best_accuracy)

C:\Users\arunkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=11, activation="relu", units=11, kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\arunkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
  if sys.path[0] == '':
C:\Users\arunkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=11, kernel_initializer="uniform")`
  del sys.path[0]
C:\Users\arunkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
  
C:\Users\arunkumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=11, kernel_initial

Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 15s 19ms/step
Epoch 1/1
800/800 [==============================] - 16s 20ms/step
Epoch 1/1
800/800 [==============================] - 16s 20ms/step
Epoch 1/1
800/800 [==============================] - 16s 20ms/step
Epoch 1/1
800/800 [==============================] - 17s 21ms/step
Epoch 1/1
800/800 [==============================] - 16s 20ms/step
Epoch 1/1
800/800 [==============================] - 16s 20ms/step
Epoch 1/1
800/800 [==============================] - 17s 21ms/step
Epoch 1/1
800/800 [==============================] - 17s 21ms/

In [ ]:
# This is how we will get perfect combination of hyper parameters values
# We can choose those values for test data predictions
# I was abort excution because it is taking more thank a day in my local machine